In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import geopandas as gpd
import copy
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, grangercausalitytests

In [ ]:
shapefile = '/content/drive/MyDrive/tl_2012_us_state/tl_2012_us_state.shp'
gdf = gpd.read_file(shapefile)

In [ ]:
os.listdir('/content/drive/MyDrive/Datathon Data')

In [ ]:
all_dfs = {}
for file in os.listdir('/content/drive/MyDrive/Datathon Data'):
  if file.endswith('.csv'):
    df = pd.read_csv(f'/content/drive/MyDrive/Datathon Data/{file}')
    all_dfs[file] = df

<ipython-input-4-fc3d682e24bb>:4: DtypeWarning: Columns (18,19,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'/content/drive/MyDrive/Datathon Data/{file}')


In [ ]:
all_dfs['acs_5yr_est_selected_economic_characteristics_2010-2022.csv']

In [ ]:
all_dfs.keys()

dict_keys(['Nutrition_Physical_Activity_and_Obesity_Data.csv', 'all_stock_and_etfs.csv', 'all_commodities.csv', 'Meat_Stats_Meat_Production.csv', 'Meat_Stats_Slaughter_Counts.csv', 'Meat_Stats_Slaughter_Weights.csv', 'Meat_Stats_Cold_Storage.csv', 'stock_descriptions.csv', 'acs_5yr_est_selected_economic_characteristics_2010-2022.csv', 'state_dollar_ag.csv'])

In [ ]:
all_obesity_questions = list(all_dfs['Nutrition_Physical_Activity_and_Obesity_Data.csv']['Question'].unique())
all_years = list(all_dfs['Nutrition_Physical_Activity_and_Obesity_Data.csv']['YearStart'].unique())

In [ ]:
def generate_chloropleth(question, year):
  question_data = all_dfs['Nutrition_Physical_Activity_and_Obesity_Data.csv'][(all_dfs['Nutrition_Physical_Activity_and_Obesity_Data.csv'].loc[:,'Question'] == question) & (all_dfs['Nutrition_Physical_Activity_and_Obesity_Data.csv'].loc[:,'YearStart'] == year) & (all_dfs['Nutrition_Physical_Activity_and_Obesity_Data.csv'].loc[:,'StratificationCategoryId1'] == 'OVR')]
  vals = question_data.loc[:,["LocationAbbr", "Data_Value"]].groupby("LocationAbbr").mean()
  merged = gdf.merge(vals, left_on='STUSPS', right_on='LocationAbbr')
  merged = merged.to_crs(epsg=4326)
  fig = plot_heatmap(merged, question, year)
  return fig

In [ ]:
def plot_heatmap(geodf, question, year):
  # Create the interactive heatmap
  fig = px.choropleth_mapbox(
      geodf,
      geojson=geodf.__geo_interface__,
      locations='STUSPS',
      color='Data_Value',
      color_continuous_scale="OrRd",
      mapbox_style="carto-positron",
      featureidkey="properties.STUSPS",
      center={"lat": 37.0902, "lon": -95.7129},
      zoom=3,
      hover_name='NAME',
      hover_data={'Data_Value': True}
  )

  fig.update_geos(fitbounds="locations")
  fig.update_layout(title=f"{question} in {year}")
  return fig

In [ ]:
all_obesity_questions

['Percent of students in grades 9-12 who achieve 1 hour or more of moderate-and/or vigorous-intensity physical activity daily',
 'Percent of students in grades 9-12 who have obesity',
 'Percent of students in grades 9-12 who participate in daily physical education',
 'Percent of students in grades 9-12 who have an overweight classification',
 'Percent of students in grades 9-12 who drank regular soda/pop at least one time per day',
 'Percent of students in grades 9-12 who consume vegetables less than 1 time daily',
 'Percent of students in grades 9-12 who consume fruit less than 1 time daily',
 'Percent of students in grades 9-12 watching 3 or more hours of television each school day',
 'Percent of adults who engage in no leisure-time physical activity',
 'Percent of adults aged 18 years and older who have obesity',
 'Percent of adults aged 18 years and older who have an overweight classification',
 'Percent of adults who achieve at least 300 minutes a week of moderate-intensity aerobi

In [ ]:
all_years = list(sorted(all_years))

In [ ]:
fig3 = generate_chloropleth('Percent of students in grades 9-12 watching 3 or more hours of television each school day', all_years[-4])
fig3.show()

In [ ]:
fig2 = generate_chloropleth(all_obesity_questions[1], all_years[-4])
fig2.show()

In [ ]:
fig = generate_chloropleth(all_obesity_questions[3], all_years[-4])
fig.show()

In [ ]:
all_dfs['Meat_Stats_Slaughter_Counts.csv']

In [ ]:
all_dfs['Meat_Stats_Meat_Production.csv']

In [ ]:
all_dfs['acs_5yr_est_selected_economic_characteristics_2010-2022.csv']

In [ ]:
all_dfs['Meat_Stats_Slaughter_Weights.csv']

In [ ]:
def plot_all_animal_stats(inspection_type = 'Federally Inspected', weight_type = 'Dressed'):
  animals = copy.deepcopy(all_dfs['Meat_Stats_Meat_Production.csv'])
  animals['Date'] = pd.to_datetime(animals['Date'], format='%b-%Y')
  animals = animals.sort_values(by='Date')

  animals['Production'] = animals['Production'].str.replace(',', '').astype(float)
  animals['Production'] = animals['Production'].astype(float)

  animals = animals[animals['Commercial or Federally Inspected'] == inspection_type]
  meat_production = px.line(animals, x="Date", y="Production", title="Meat production by Animal", color = "Animal")


  counts = copy.deepcopy(all_dfs['Meat_Stats_Slaughter_Counts.csv'])
  counts['Date'] = pd.to_datetime(counts['Date'], format='%b-%Y')
  counts = counts.sort_values(by='Date')

  counts['Count'] = counts['Count'].str.replace(',', '').astype(float)
  counts['Count'] = counts['Count'].astype(float)

  counts = counts[counts['Commercial_Or_Federally_Inspected'] == inspection_type]
  slaughter_counts = px.line(counts, x="Date", y="Count", title="Slaughter Counts by Animal", color = "Animal")

  weights = copy.deepcopy(all_dfs['Meat_Stats_Slaughter_Weights.csv'])
  weights['Date'] = pd.to_datetime(weights['Date'], format='%b-%Y')
  weights = weights.sort_values(by='Date')

  weights['Weight'] = weights['Weight'].str.replace(',', '').astype(float)
  weights['Weight'] = weights['Weight'].astype(float)

  weights = weights[(weights['Commercial_Or_Federally_Inspected'] == inspection_type) & (weights['Live_Or_Dressed'] == weight_type)]
  slaughter_weights = px.line(weights, x="Date", y="Weight", title="Slaughter Weight by Animal", color = "Animal")

  storage = copy.deepcopy(all_dfs['Meat_Stats_Cold_Storage.csv'])
  storage['Date'] = pd.to_datetime(storage['Date'], format='%b-%Y')
  storage = storage.sort_values(by='Date')
  cold_storage = px.line(storage, x="Date", y="Weight", title="Cold Storage by Animal", color = "Animal")
  return meat_production, slaughter_counts, slaughter_weights, cold_storage
meat_production, slaughter_counts, slaughter_weights, cold_storage = plot_all_animal_stats()
slaughter_weights.show()
cold_storage.show()
meat_production.show()
slaughter_counts.show()

In [ ]:
meat_production.show()

In [ ]:
all_dfs.keys()

dict_keys(['Nutrition_Physical_Activity_and_Obesity_Data.csv', 'all_stock_and_etfs.csv', 'all_commodities.csv', 'Meat_Stats_Meat_Production.csv', 'Meat_Stats_Slaughter_Counts.csv', 'Meat_Stats_Slaughter_Weights.csv', 'Meat_Stats_Cold_Storage.csv', 'stock_descriptions.csv', 'acs_5yr_est_selected_economic_characteristics_2010-2022.csv'])

In [ ]:
all_dfs['stock_descriptions.csv']

In [ ]:
all_dfs['all_stock_and_etfs.csv']

In [ ]:
def plot_commodity_prices():
  commodities = copy.deepcopy(all_dfs['all_commodities.csv'])
  commodities['Date-Time'] = pd.to_datetime(commodities['Date-Time'], format='%Y-%m-%d')
  commodities['Unit'] = commodities['Unit'].fillna('Dollar per Metric Ton')
  commodities['Commodity'] = commodities['Commodity'].fillna('Corn')
  commodities = commodities.sort_values(by='Date-Time')
  prices = px.line(commodities, x="Date-Time", y="Value", title="Prices for different commodities", color = "Commodity", hover_data={'Unit': True})
  return prices

def plot_stock_prices():
  stocks = copy.deepcopy(all_dfs['all_stock_and_etfs.csv'])
  stocks['Date-Time'] = pd.to_datetime(stocks['Date-Time'], format='%Y-%m-%d')
  stocks = stocks.sort_values(by='Date-Time')
  prices = px.line(stocks, x="Date-Time", y="Close", title="Prices for different stocks", color = "Ticker_Symbol")
  return prices
prices = plot_commodity_prices()
stocks = plot_stock_prices()
prices.show()
stocks.show()

In [ ]:
all_dfs['all_commodities.csv'][all_dfs['all_commodities.csv']['Commodity'] == 'Corn']

In [ ]:
all_dfs['all_commodities.csv']

In [ ]:

def check_stationarity(series):
  result = adfuller(series)
  print('ADF Statistic: %f' % result[0])
  print('p-value: %f' % result[1])

In [ ]:
tmp = all_dfs['all_commodities.csv'].sort_values(by='Date-Time')
check_stationarity(tmp[tmp['Commodity'] == 'Sugar'].loc[:,'Value'])

ADF Statistic: -2.275770
p-value: 0.179902


In [ ]:
commodities_copy = copy.deepcopy(all_dfs['all_commodities.csv'])
commodities_copy['Unit'] = commodities_copy['Unit'].fillna('Dollar per Metric Ton')
commodities_copy['Commodity'] = commodities_copy['Commodity'].fillna('Corn')
Corn = commodities_copy[commodities_copy['Commodity'] == 'Corn']
Corn.rename(columns={'Date-Time': 'Date'}, inplace=True)
Corn['Date'] = pd.to_datetime(Corn['Date'])

Coffee = commodities_copy[commodities_copy['Commodity'] == 'Coffee']
Coffee.rename(columns={'Date-Time': 'Date'}, inplace=True)
Coffee['Date'] = pd.to_datetime(Coffee['Date'])

Sugar = commodities_copy[commodities_copy['Commodity'] == 'Sugar']
Sugar.rename(columns={'Date-Time': 'Date'}, inplace=True)
Sugar['Date'] = pd.to_datetime(Sugar['Date'])

In [ ]:
Coffee = Coffee.set_index('Date')
Corn = Corn.set_index('Date')
Sugar = Sugar.set_index('Date')

In [ ]:
corn_sugar = Corn.join(Sugar, lsuffix = "-corn", rsuffix = "-sugar")

In [ ]:
def get_aggregate_meat_production():
  meat_production = copy.deepcopy(all_dfs['Meat_Stats_Meat_Production.csv'])

  meat_production['Production'] = pd.to_numeric(meat_production['Production'].str.replace(',', ''), errors='coerce')
  aggregated_df = meat_production.groupby('Date', as_index=False)['Production'].sum()
  aggregated_df['Date'] = pd.to_datetime(aggregated_df['Date'])
  aggregated_df = aggregated_df.sort_values(by='Date')
  print(aggregated_df)
  return aggregated_df

def clean_series(column_name, df):
  aggregated = copy.deepcopy(df)
  rolling_mean = aggregated[column_name].rolling(window=3).mean()
  deviation = aggregated[column_name] - rolling_mean
  std_dev = deviation.std()
  outliers = aggregated[(deviation.abs() > 2 * std_dev)]
  cleaned_aggregated = aggregated.drop(outliers.index)
  return cleaned_aggregated

In [ ]:
aggregated = get_aggregate_meat_production()
cleaned_aggregated = clean_series('Production', aggregated)
cleaned_aggregated['Date'] = pd.to_datetime(cleaned_aggregated['Date'])
cleaned_aggregated_dff = cleaned_aggregated['Production'].diff().dropna()
cleaned_aggregated_dff = cleaned_aggregated_dff.reset_index(drop=True)
cleaned_aggregated = cleaned_aggregated.set_index('Date')

In [ ]:
total = cleaned_aggregated.join(Corn, lsuffix = "Meat", rsuffix = "Corn", how = "inner")

In [ ]:
total

In [ ]:
print(check_stationarity(total['Production'].diff().dropna()))

In [ ]:
print(check_stationarity(corn_sugar['Value-corn'].diff().dropna()))
print(check_stationarity(corn_sugar['Value-sugar'].diff().dropna()))
# P-value of 0.2, Corn prices don't have strong corr with sugar prices
print(grangercausalitytests(corn_sugar[['Value-corn', 'Value-sugar']].diff().dropna(), maxlag=8))

In [ ]:
print(check_stationarity(total['Production'].diff().dropna()))
print(check_stationarity(total['Value'].diff().dropna()))
# P-value of 0.2, Corn prices don't have strong corr with meat production levels
print(grangercausalitytests(total[['Production', 'Value']].diff().dropna(), maxlag=4))

In [ ]:
all_dfs['Nutrition_Physical_Activity_and_Obesity_Data.csv'][(all_dfs['Nutrition_Physical_Activity_and_Obesity_Data.csv']['Question'] == 'Percent of students in grades 9-12 who have obesity') & (all_dfs['Nutrition_Physical_Activity_and_Obesity_Data.csv']['LocationAbbr'] == 'TX') & (all_dfs['Nutrition_Physical_Activity_and_Obesity_Data.csv']['StratificationCategory1'] == 'Total')]

In [ ]:
all_dfs['Nutrition_Physical_Activity_and_Obesity_Data.csv']['Question'].unique()

In [ ]:
all_dfs['Nutrition_Physical_Activity_and_Obesity_Data.csv'][(all_dfs['Nutrition_Physical_Activity_and_Obesity_Data.csv']['Question'] == 'Percent of adults aged 18 years and older who have obesity') & (all_dfs['Nutrition_Physical_Activity_and_Obesity_Data.csv']['LocationAbbr'] == 'CA') & (all_dfs['Nutrition_Physical_Activity_and_Obesity_Data.csv']['StratificationCategory1'] == 'Total')]

In [ ]:
all_dfs['Nutrition_Physical_Activity_and_Obesity_Data.csv'][all_dfs['Nutrition_Physical_Activity_and_Obesity_Data.csv']['Question'] == 'Percent of adults who report consuming vegetables less than one time daily']

In [ ]:
all_dfs.keys()

dict_keys(['Nutrition_Physical_Activity_and_Obesity_Data.csv', 'all_stock_and_etfs.csv', 'all_commodities.csv', 'Meat_Stats_Meat_Production.csv', 'Meat_Stats_Slaughter_Counts.csv', 'Meat_Stats_Slaughter_Weights.csv', 'Meat_Stats_Cold_Storage.csv', 'stock_descriptions.csv', 'acs_5yr_est_selected_economic_characteristics_2010-2022.csv'])

In [ ]:
all_dfs['acs_5yr_est_selected_economic_characteristics_2010-2022.csv']['Label (Grouping)'].unique()

In [ ]:
all_dfs['acs_5yr_est_selected_economic_characteristics_2010-2022.csv']